# Initial Naive Classification Approaches

The following approaches are naive because they tend to perform well where the data contains linear relationships, and handwriting can be quite unique per person. These approaches tend to struggle with non-linear relationships and complex patterns generally, and it will be interesting to see their performance on the dataset.

- Logistic Regression Classifier
- Naive Bayes Classifier
- Decision Trees Classifier
- K-Nearest Neighbors Classifier
- Support Vector Machines Classifier

## Load Dependencies and Dataset

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')